In [1]:
from collections import defaultdict

from pathlib import Path
import os
import sys
import json
import pickle
#os.environ['CUDA_VISIBLE_DEVICES']='0'  # don't occupy GPU, inference can work on CPU
import torch
import logging
import numpy as np
from tqdm import tqdm
import pandas as pd
from typing import List, Dict, Tuple

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from flair.embeddings import BertEmbeddings
from flair.data import Sentence
from sklearn.pipeline import Pipeline

from python_ev.dataset import load_parts, load_df
from vectorizers.bert_boris.bert_base_spwsi import BertBaseSPWSI
from subtask1_solutions.base_solution import Subtask1BaseVectorizerFactory
import flair

02/07/2019 13:20:50 - INFO - summarizer.preprocessing.cleaner -   'pattern' package not found; tag filters are not available for English
/mnt/raid/v.omelchenko/anaconda/envs/rejection_clean/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [20]:
from vectorizers.bert_boris.bert_flair_from_notebook_vectorizer import BertFlairFromNotebookVectorizer, test

vectorizer = BertFlairFromNotebookVectorizer(tokens_field='FORM',
                                             model='bert-base-multilingual-cased', 
                                             add_bias=True, 
                                             normalize_out_embs=True, 
                                             pattern='_',
                                             prediction_cutoff=20,
                                             lemmatize_type=None,
                                             verbose=True, 
                                             exclude_lemma=False)

Hyperparameters:
Add_bias: True
Normalize_out_embs: True
Pattern: _



In [21]:
sent = 'Ей он рассказывает одно, а нам - ток совершенно другое'.split(' ')
key = vectorizer.get_inst_id(7, 'dsdsg')
dct = {key: (sent, 7, '')}

In [22]:
vectorizer.predict_sent_substitute_representatives(dct)



generate_vecs: 0it [00:00, ?it/s]


Vectorization:   0%|          | 0/1 [00:00<?, ?it/s]


Vectorization: 100%|██████████| 1/1 [00:00<00:00, 19.70it/s]

generate_vecs: 1it [00:00, 17.55it/s]

predict_word_scores:   0%|          | 0/1 [00:00<?, ?it/s]

INPUT TO BERT: [CLS] | е | ##и | он | рассказывает | одно | , | а | нам | - | >>[MASK]<< | совершенно | другое | [SEP]




predict_word_scores: 100%|██████████| 1/1 [00:01<00:00,  1.56s/it]



{'7.dsdsg': ['то и в это на - во о он что ему за не у , говорит есть та как только']}

In [69]:
def gen_list_of_inputs(sent, tok = 'MASK', lemma = ''):
    tokens = sent.split(' ')
    res = []
    for i in range(0, len(tokens) + 1):
        tokens = sent.split(' ')
        tokens.insert(i, tok)
        res.append({str(i):(tokens, i, lemma)})
    return res

In [106]:
#sent = 'Акциз на бензин в США при розничных продажах составляет 10 центов за литр, на дизельное топливо - 12 центов.'
#sent = 'Я думаю, она вскоре будет на ногах и через несколько дней на пути к выздоровлению.'
#sent = 'Работа с двухбайтовыми наборами символов — просто кошмар для программиста, так как часть их состоит из одного байта, а часть — из двух.'
key = 'sent.sent'

bert_inps = gen_list_of_inputs(sent)

In [107]:
vectorizer = BertFlairFromNotebookVectorizer(tokens_field='FORM',
                                             model='bert-base-multilingual-cased', 
                                             add_bias=True, 
                                             normalize_out_embs=True, 
                                             pattern='_',
                                             prediction_cutoff=20,
                                             lemmatize_type=None,
                                             verbose=False, 
                                             exclude_lemma=False)

results = []
for bert_inp in bert_inps:
    res = vectorizer.predict_sent_substitute_representatives(bert_inp)
    results.append((list(bert_inp.values())[0][0], list(res.values())[0]))



generate_vecs: 0it [00:00, ?it/s]

generate_vecs: 1it [00:00, 20.43it/s]

predict_word_scores:   0%|          | 0/1 [00:00<?, ?it/s]

predict_word_scores: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it]



generate_vecs: 0it [00:00, ?it/s]

generate_vecs: 1it [00:00, 29.27it/s]

predict_word_scores:   0%|          | 0/1 [00:00<?, ?it/s]

predict_word_scores: 100%|██████████| 1/1 [00:01<00:00,  1.72s/it]



generate_vecs: 0it [00:00, ?it/s]

generate_vecs: 1it [00:00, 32.15it/s]

predict_word_scores:   0%|          | 0/1 [00:00<?, ?it/s]

predict_word_scores: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it]



generate_vecs: 0it [00:00, ?it/s]

generate_vecs: 1it [00:00, 31.20it/s]

predict_word_scores:   0%|          | 0/1 [00:00<?, ?it/s]

predict_word_scores: 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]



generate_vecs: 0it [00:00, ?it/s]

generate_vecs: 1it [00:00, 31.80it/s]

predict_word_scores:   0%|          | 0/1 [00:00<?, ?it/s]

predict_word_scores: 100%|██████████| 1/1 [00:01

In [59]:
results

[(['MASK',
   'Акциз',
   'на',
   'бензин',
   'в',
   'США',
   'при',
   'розничных',
   'продажах',
   'составляет',
   '10',
   'центов',
   'за',
   'литр,',
   'на',
   'дизельное',
   'топливо',
   '-',
   '12',
   'центов.'],
  ['На В К С . Средний За А При Р , - 1 Однако Д Так Поэтому ; М П']),
 (['Акциз',
   'MASK',
   'на',
   'бензин',
   'в',
   'США',
   'при',
   'розничных',
   'продажах',
   'составляет',
   '10',
   'центов',
   'за',
   'литр,',
   'на',
   'дизельное',
   'топливо',
   '-',
   '12',
   'центов.'],
  ['. , ; : ##ло ##ля ##л ##ка - ##ль ##ла ##ли ##лов ##ле ##р воды ) на ##ление ##лы']),
 (['Акциз',
   'на',
   'MASK',
   'бензин',
   'в',
   'США',
   'при',
   'розничных',
   'продажах',
   'составляет',
   '10',
   'центов',
   'за',
   'литр,',
   'на',
   'дизельное',
   'топливо',
   '-',
   '12',
   'центов.'],
  ['на новый 1 2 - 4 первый ##ный : , в 3 . 100 5 6 и один завод 12']),
 (['Акциз',
   'на',
   'бензин',
   'MASK',
   'в',
   'США',

In [110]:
top_elems = 5

for elem in results: # ([elems], [str])
    fst = elem[0]
    snd = elem[1][0].split(' ')[:top_elems]
    snd = [elem for elem in snd if elem.isalpha()]
    if (not set(fst).isdisjoint(snd)):
        print(fst)
        print(snd)
        print('------------')

['Работа', 'с', 'двухбайтовыми', 'наборами', 'MASK', 'символов', '—', 'просто', 'кошмар', 'для', 'программиста,', 'так', 'как', 'часть', 'их', 'состоит', 'из', 'одного', 'байта,', 'а', 'часть', '—', 'из', 'двух.']
['для']
------------
['Работа', 'с', 'двухбайтовыми', 'наборами', 'символов', 'MASK', '—', 'просто', 'кошмар', 'для', 'программиста,', 'так', 'как', 'часть', 'их', 'состоит', 'из', 'одного', 'байта,', 'а', 'часть', '—', 'из', 'двух.']
['для', 'в']
------------
['Работа', 'с', 'двухбайтовыми', 'наборами', 'символов', '—', 'просто', 'кошмар', 'для', 'MASK', 'программиста,', 'так', 'как', 'часть', 'их', 'состоит', 'из', 'одного', 'байта,', 'а', 'часть', '—', 'из', 'двух.']
['для', 'всего', 'его', 'у', 'работы']
------------
['Работа', 'с', 'двухбайтовыми', 'наборами', 'символов', '—', 'просто', 'кошмар', 'для', 'программиста,', 'MASK', 'так', 'как', 'часть', 'их', 'состоит', 'из', 'одного', 'байта,', 'а', 'часть', '—', 'из', 'двух.']
['как', 'и', 'только']
------------
['Работа'